In [1]:
import os
import sys
os.environ["SPARK_HOME"]="/home/user1/trainig_material/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")
import json
import time 
from pyspark.sql import SparkSession, Row, SQLContext
from kafka import KafkaConsumer
consumer = KafkaConsumer('twitter', group_id=None, auto_offset_reset='latest', bootstrap_servers=['localhost:9092'], value_deserializer=lambda m: json.loads(m)) 

avg_hashtag = []
minutes = []

rows = []

 

timer = time.time() + 30
for msg in consumer:
    row = json.loads(json.dumps(msg.value))
    if 'entities' in row:
        if len(row['entities']['hashtags']) > 0:
            hashtag_text = row['entities']['hashtags'][0]['text']
        else:
            continue
            hashtag_text = row['entities']['hashtags']
        #print(hashtag_text)
        #Set up stream format
        row = Row(TweetID=row['id'], Text=row['text'], UserID=row['user']['id'], Hashtags=hashtag_text, Event_time=row['created_at'])
        rows.append(row)
        #print(msg.value)
        print(row)
    if (time.time() >= timer):
        break
    timer = timer-1

 
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

tweetsdf = spark.createDataFrame(rows)
tweetsdf.show()

Row(Event_time='Tue Jul 27 19:18:29 +0000 2021', Hashtags='timetoshine', Text='Good luck to all the men and women for tomorrow’s TT in 🇯🇵 but especially Team 🇺🇸 !!  #timetoshine #olympics… https://t.co/eAO2FlrGqe', TweetID=1420101301690634245, UserID=38980965)
Row(Event_time='Tue Jul 27 19:18:29 +0000 2021', Hashtags='SimoneBiles', Text='RT @DineshDSouza: You can count on the sniveling US media to turn vice into virtue. Now they’re praising #SimoneBiles for her “courageous”…', TweetID=1420101302114213889, UserID=2188767095)
Row(Event_time='Tue Jul 27 19:18:29 +0000 2021', Hashtags='DriveYouHome', Text='RT @OnAirJRDN: The Olympics are going down so naturally I had to talk to @JacksonWang852 👀 Interview soon..\n#DriveYouHome \n#TeamWang https:…', TweetID=1420101302445432833, UserID=1415122150730604545)
Row(Event_time='Tue Jul 27 19:18:30 +0000 2021', Hashtags='Olympics', Text='This here is gold medal deflection. #Olympics', TweetID=1420101306912542727, UserID=14197213)
Row(Event_time='Tu

In [5]:
from pyspark.sql.functions import desc
tweetsdf.printSchema()

#tweetsdf.sort("Hashtags").show()

root
 |-- Event_time: string (nullable = true)
 |-- Hashtags: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- TweetID: long (nullable = true)
 |-- UserID: long (nullable = true)



In [4]:


timer = time.time() + 600
for msg in consumer:
    row = json.loads(json.dumps(msg.value))
    if 'entities' in row:
        if len(row['entities']['hashtags']) > 0:
            hashtag_text = row['entities']['hashtags'][0]['text']
        else:
            continue
            hashtag_text = row['entities']['hashtags']
        #print(hashtag_text)
        #Set up stream format
        row = Row(TweetID=row['id'], Text=row['text'], UserID=row['user']['id'], Hashtags=hashtag_text, Event_time=row['created_at'])
        rows.append(row)
        #print(msg.value)
        print(row)
    if (time.time() >= timer):
        break
    timer = timer-1

tweetsdf2 = spark.createDataFrame(rows)
tweetsdf2.show(1000)

Row(Event_time='Tue Jul 27 19:18:42 +0000 2021', Hashtags='Olympics', Text="RT @enews: BREAKING: Simone Biles has withdrawn from the women's gymnastics team final at the Tokyo #Olympics. https://t.co/ODgPXzzePY http…", TweetID=1420101355495112704, UserID=779828790)
Row(Event_time='Tue Jul 27 19:18:42 +0000 2021', Hashtags='NGA', Text="RT @Olympics: Coach Mike Brown has HUGE ambitions for his time in charge of the men's basketball team from @NigeriaBasket #NGA \n\nHis team t…", TweetID=1420101356170448896, UserID=1051376509078913024)
Row(Event_time='Tue Jul 27 19:18:42 +0000 2021', Hashtags='olympics', Text='Mr. Chili and Manny playing doubles!\n🎾\n\n.\n\n🎾\n\n.\n\n🎾\n\n.\n\n🎾\n#olympics #mannybolo #reeftank #reefaquarium… https://t.co/P0rlxMrTyL', TweetID=1420101356979998724, UserID=2295534036)
Row(Event_time='Tue Jul 27 19:18:43 +0000 2021', Hashtags='SimoneBiles', Text='RT @Alex_Verbeek: 🤸\u200d♀️ \n\nWatch Simone Biles in extreme slow motion. \n\n#SimoneBiles #Olympics #OlympicGames

Row(Event_time='Tue Jul 27 19:19:39 +0000 2021', Hashtags='Surfing', Text='RT @Olympics: Shredding her way to bronze!\n\nTsuzuki Amuro is the first female surfer to win an Olympic medal!\n\n#Surfing @isasurfing @Japan_…', TweetID=1420101594587205639, UserID=1077992828)
Row(Event_time='Tue Jul 27 19:19:40 +0000 2021', Hashtags='FreebieFriday', Text='RT @DenmanBrush: To celebrate the Olympics starting today, this #FreebieFriday we are giving away this exclusive D3 Olympics brush to one l…', TweetID=1420101598924115975, UserID=721318341823111168)
Row(Event_time='Tue Jul 27 19:19:40 +0000 2021', Hashtags='Beijing', Text='RT @IndoPac_Info: Diplomatic Boycott of #Beijing Olympics: Now, Make It a Reality\n\n#UK parliamentarians voted unanimously. What will the go…', TweetID=1420101602027851777, UserID=1384573120070045697)
Row(Event_time='Tue Jul 27 19:19:42 +0000 2021', Hashtags='TeamGB', Text='I can’t stop watching this!! #TeamGB #Tokyo2020 #Olympics', TweetID=1420101611276410882, UserID=202

Row(Event_time='Tue Jul 27 19:20:27 +0000 2021', Hashtags='TeamUSA', Text='RT @USARugby: Once again. \n\nLOUDER for the people in the back, @MikeFriday09. \n\n#TeamUSA #Olympics https://t.co/JY3rNRtC1M', TweetID=1420101796052275209, UserID=2840077301)
Row(Event_time='Tue Jul 27 19:20:27 +0000 2021', Hashtags='Olympics', Text='RT @leahmcelrath: I stand in solidarity with Simone Biles doing what is necessary to protect her mental health.\n\n#Olympics \n#Tokyo2020 http…', TweetID=1420101799445286914, UserID=436237971)
Row(Event_time='Tue Jul 27 19:20:28 +0000 2021', Hashtags='TUN', Text="RT @JoySportsGH: Look. At. This.\n\nAhmed Hafnaoui's family back in #TUN react as the teenager wins 400m #swimming gold at the #Olympics. \n\n❤…", TweetID=1420101802897383427, UserID=142538929)
Row(Event_time='Tue Jul 27 19:20:29 +0000 2021', Hashtags='olympics', Text='RT @chipgoines: Six years ago today, the @boston2024 #olympics bid was cancelled and Boston became the first city since Denver in 1976 to 

Row(Event_time='Tue Jul 27 19:21:22 +0000 2021', Hashtags='gold', Text='RT @Olympics: Another #gold medal for Milica Mandic of #SRB in the women’s +67kg #Taekwondo! \n\n#Gold London 2012\n#Gold Tokyo 2020\n\n#Stronge…', TweetID=1420102029654040578, UserID=1408388922166685696)
Row(Event_time='Tue Jul 27 19:21:23 +0000 2021', Hashtags='gymnasticstwitter', Text='#gymnasticstwitter once again\n"There is so much to take away here about gymnastics and American-ness and the percep… https://t.co/rpWJnviS2Y', TweetID=1420102034531946496, UserID=3702381389)
Row(Event_time='Tue Jul 27 19:21:24 +0000 2021', Hashtags='PHI', Text='RT @Olympics: History. Made.\n\nHidilyn Diaz has won the first ever gold medal for Philippines!\n\n#PHI #Gold #PHI #Gold #PHI #Gold https://t.c…', TweetID=1420102037618847745, UserID=727865699129942018)
Row(Event_time='Tue Jul 27 19:21:26 +0000 2021', Hashtags='Tokyo2020', Text='RT @KirenRijiju: Please note down match schedule of India at #Tokyo2020 Olympics on 28th July, 

Row(Event_time='Tue Jul 27 19:22:24 +0000 2021', Hashtags='Olympics', Text='RT @ChinaAmbSA: Congratulations! The #Olympics has long been an occasion for national joy.  "Faster, Higher, Stronger - Together".\xa0Way to g…', TweetID=1420102286668296193, UserID=1115112938841333760)
Row(Event_time='Tue Jul 27 19:22:24 +0000 2021', Hashtags='crypto', Text='@Plinz If there is a backdoor or " kill switch" it isn\'t #crypto its just another digital dollar, like all fraction… https://t.co/2NfODp0NWI', TweetID=1420102287507165184, UserID=1310016742815539200)
Row(Event_time='Tue Jul 27 19:22:24 +0000 2021', Hashtags='Olympics', Text='RT @SadhguruJV: Dhanalakshmi, Bharat is proud of your Determination &amp; Perseverance. My Best Wishes for your Success at the #Olympics. Bless…', TweetID=1420102288362770434, UserID=1144131541959733249)
Row(Event_time='Tue Jul 27 19:22:24 +0000 2021', Hashtags='Olympics', Text='RT @emilykschrader: Simply amazing! Iranian coach Vahid Sarlak films a video with an Israe

Row(Event_time='Tue Jul 27 19:23:20 +0000 2021', Hashtags='Olympics', Text='RT @ajplus: UPDATE: Simone Biles spoke about athlete mental health after exiting an #Olympics team event, saying she did not like her perfo…', TweetID=1420102523344543750, UserID=52184716)
Row(Event_time='Tue Jul 27 19:23:20 +0000 2021', Hashtags='SimoneBiles', Text='RT @DineshDSouza: You can count on the sniveling US media to turn vice into virtue. Now they’re praising #SimoneBiles for her “courageous”…', TweetID=1420102525529862144, UserID=1413870945387925513)
Row(Event_time='Tue Jul 27 19:23:22 +0000 2021', Hashtags='TokyoOlympics', Text='RT @pushan2301: \u2066@manikabatra_TT\u2069 personal coach \u2066@sanmaytt\u2069 speaks to \u2066@ttindia\u2069 @olympics #TokyoOlympics #tabletennis https://t.co/EDUa06…', TweetID=1420102532387377153, UserID=1125428351131213824)
Row(Event_time='Tue Jul 27 19:23:23 +0000 2021', Hashtags='SimoneBiles', Text='RT @CasNegley: A must-read on #SimoneBiles by @JulietMacur. \n\nIt 

Row(Event_time='Tue Jul 27 19:24:21 +0000 2021', Hashtags='Surfing', Text="RT @Olympics: Italo Ferreira is the inaugural men's Olympic #Surfing champion!\n\n@isasurfing @TimeBrasil #BRA https://t.co/hNV6qv1Ihj", TweetID=1420102780312752137, UserID=54282006)
Row(Event_time='Tue Jul 27 19:24:24 +0000 2021', Hashtags='IOC', Text="Well...the only other option at the time was Kazakhstan...so...\n\nLet's🤞that the #IOC figures out a way to entice co… https://t.co/uvDIJ2DHvM", TweetID=1420102791452782592, UserID=1626905642)
Row(Event_time='Tue Jul 27 19:24:24 +0000 2021', Hashtags='SimoneBiles', Text='RT @TheSWDiva: Black Women prioritizing their mental health brings me so much joy. The guise of being "strong" is overrated. #SimoneBiles #…', TweetID=1420102791759024132, UserID=1104082174817505286)
Row(Event_time='Tue Jul 27 19:24:24 +0000 2021', Hashtags='Preparedness', Text='Are you ready for the #Preparedness #Olympics!?\n\nOur main event in #Earthquake preparedness is our own special bran… 

Row(Event_time='Tue Jul 27 19:24:46 +0000 2021', Hashtags='Tokyo2020', Text='RT @TeamSA2020: Introducing your #Tokyo2020 Women’s 100m Breaststroke Silver Medalist @TRSchoenmaker 😁🥈🇿🇦 \n\n📸 @AntonGeyser \n\n#TeamSA\n#Olymp…', TweetID=1420102883169738761, UserID=148475514)
Row(Event_time='Tue Jul 27 19:24:46 +0000 2021', Hashtags='olympics', Text='The uniform specifications should be for the sport, not the gender.\n\n#olympics', TweetID=1420102884537114625, UserID=3278758339)
Row(Event_time='Tue Jul 27 19:24:46 +0000 2021', Hashtags='Tokyo2020', Text='RT @Olympics: The Olympic Cauldron has been lit! 🔥\n\n@naomiosaka does the honours as the curtain rises on #Tokyo2020. \n\n#OpeningCeremony | #…', TweetID=1420102884788682752, UserID=1713558188)
Row(Event_time='Tue Jul 27 19:24:47 +0000 2021', Hashtags='TokyoOlympics', Text='RT @NBCOlympics: In softball’s first appearance at the Olympics since 2008, @TeamUSA earns silver. 🥈\n\n#TokyoOlympics x @USASoftballWNT http…', TweetID=142010289020778

In [5]:
# 7a 
tweetsdf.count()

19

In [6]:
tweetsdf2.count()

610

In [7]:
# 7b
tweetsdf.select(tweetsdf['UserID']).distinct().count()

19

In [8]:
tweetsdf2.select(tweetsdf2['UserID']).distinct().count()

511

In [11]:
# 7c
from pyspark.sql.functions import *
tweetsdf.groupBy("Hashtags").count().orderBy(desc("count")).show()

+------------+-----+
|    Hashtags|count|
+------------+-----+
|    Olympics|    5|
|   Tokyo2020|    3|
| SimoneBiles|    2|
|DriveYouHome|    1|
|CyrusAccords|    1|
|      bronze|    1|
|       ICYMI|    1|
| SimomeBiles|    1|
|Olympics2020|    1|
|         NGA|    1|
|Brisbane2032|    1|
| timetoshine|    1|
+------------+-----+



In [12]:
tweetsdf2.groupBy("Hashtags").count().orderBy(desc("count")).show(100)

+--------------------+-----+
|            Hashtags|count|
+--------------------+-----+
|            Olympics|  185|
|           Tokyo2020|   58|
|         SimoneBiles|   29|
|       TokyoOlympics|   14|
|BestWaytoStayConn...|   13|
|              silver|   12|
|                gold|   12|
|                 NGA|    8|
|              bronze|    8|
|        CyrusAccords|    7|
|        DriveYouHome|    6|
|             Olympic|    6|
|              TeamGB|    6|
|            olympics|    6|
|                 JPN|    6|
|                 USA|    6|
|       weightlifting|    5|
|    BlackLivesMatter|    5|
|                 GBR|    5|
|             Surfing|    5|
|                 PHI|    4|
|                 SUI|    4|
|         dreamfanart|    4|
|        OlympicGames|    4|
|         bbcolympics|    4|
|                 CAN|    3|
|    StrongerTogether|    3|
|               USWNT|    3|
|                 CHN|    3|
|              Silver|    3|
|       FreebieFriday|    3|
|             

In [13]:
#7d
tweetsdf2.groupBy("Hashtags").count().orderBy(desc("count")).show(1000)
numTweets = tweetsdf2.select(tweetsdf2['TweetID']).count()
print(numTweets)

+--------------------+-----+
|            Hashtags|count|
+--------------------+-----+
|            Olympics|  185|
|           Tokyo2020|   58|
|         SimoneBiles|   29|
|       TokyoOlympics|   14|
|BestWaytoStayConn...|   13|
|              silver|   12|
|                gold|   12|
|              bronze|    8|
|                 NGA|    8|
|        CyrusAccords|    7|
|             Olympic|    6|
|              TeamGB|    6|
|                 USA|    6|
|        DriveYouHome|    6|
|                 JPN|    6|
|            olympics|    6|
|             Surfing|    5|
|    BlackLivesMatter|    5|
|                 GBR|    5|
|       weightlifting|    5|
|         bbcolympics|    4|
|                 PHI|    4|
|                 SUI|    4|
|        OlympicGames|    4|
|         dreamfanart|    4|
|                 CAN|    3|
|                 CZE|    3|
|           Toyko2020|    3|
|                 GER|    3|
|    StrongerTogether|    3|
|               Assam|    3|
|             

In [14]:
for i in avg_hashtag: 
    i.createOrReplaceTempView('minute')
    spark.sql('select count(Hashtags) as Count from minute where Hashtags = "Olympics"').show()

In [15]:
tweetsdf2.select(tweetsdf2['Event_time']).show(1000, False)

+------------------------------+
|Event_time                    |
+------------------------------+
|Tue Jul 27 19:18:29 +0000 2021|
|Tue Jul 27 19:18:29 +0000 2021|
|Tue Jul 27 19:18:29 +0000 2021|
|Tue Jul 27 19:18:30 +0000 2021|
|Tue Jul 27 19:18:30 +0000 2021|
|Tue Jul 27 19:18:30 +0000 2021|
|Tue Jul 27 19:18:31 +0000 2021|
|Tue Jul 27 19:18:31 +0000 2021|
|Tue Jul 27 19:18:34 +0000 2021|
|Tue Jul 27 19:18:35 +0000 2021|
|Tue Jul 27 19:18:36 +0000 2021|
|Tue Jul 27 19:18:37 +0000 2021|
|Tue Jul 27 19:18:37 +0000 2021|
|Tue Jul 27 19:18:38 +0000 2021|
|Tue Jul 27 19:18:38 +0000 2021|
|Tue Jul 27 19:18:39 +0000 2021|
|Tue Jul 27 19:18:39 +0000 2021|
|Tue Jul 27 19:18:39 +0000 2021|
|Tue Jul 27 19:18:41 +0000 2021|
|Tue Jul 27 19:18:42 +0000 2021|
|Tue Jul 27 19:18:42 +0000 2021|
|Tue Jul 27 19:18:42 +0000 2021|
|Tue Jul 27 19:18:43 +0000 2021|
|Tue Jul 27 19:18:44 +0000 2021|
|Tue Jul 27 19:18:45 +0000 2021|
|Tue Jul 27 19:18:45 +0000 2021|
|Tue Jul 27 19:18:46 +0000 2021|
|Tue Jul 2

In [16]:
# 7e
from pyspark.sql.functions import substring
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
#tweetsdf2.select(tweetsdf2['Event_time']).orderBy(desc('Event_time')).show(truncate=False)

#time=tweetsdf2.select(substring(tweetsdf2.Event_time,12,8))
#time.show()
#df.withColumn(tweetsdf2.Event_time, F.regexp_replace(F.regexp_replace(regexp_replace(tweetsdf2, ))))
time = tweetsdf2.withColumn('Event_time', regexp_replace('Event_time', ':', ''))
time.show(1000)
time=time.select(substring(time.Event_time,12,6))
time=time.withColumn('substring(Event_time, 12, 6)', time['substring(Event_time, 12, 6)'].cast(IntegerType()))
first = time.head()
print(first[0])
last = time.orderBy(desc('substring(Event_time, 12, 6)'))
last = last.head()
print(last[0])

#avg amount of tweets/minute
print(numTweets/(last[0]-first[0])/1000/60)

def movingAvg(hashtag):
    for i in range(tweetsdf.count()):
        continue
        
#movingAvg('h')
    

+--------------------+--------------------+--------------------+-------------------+-------------------+
|          Event_time|            Hashtags|                Text|            TweetID|             UserID|
+--------------------+--------------------+--------------------+-------------------+-------------------+
|Tue Jul 27 191829...|         timetoshine|Good luck to all ...|1420101301690634245|           38980965|
|Tue Jul 27 191829...|         SimoneBiles|RT @DineshDSouza:...|1420101302114213889|         2188767095|
|Tue Jul 27 191829...|        DriveYouHome|RT @OnAirJRDN: Th...|1420101302445432833|1415122150730604545|
|Tue Jul 27 191830...|            Olympics|This here is gold...|1420101306912542727|           14197213|
|Tue Jul 27 191830...|         SimomeBiles|🚨🗣—#SimomeBiles...|1420101306966974467|1382114603178496001|
|Tue Jul 27 191830...|        CyrusAccords|#CyrusAccords com...|1420101308292403204|          231069949|
|Tue Jul 27 191831...|            Olympics|RT @JeffDLowe:

192454
1.6266666666666668e-05
